In [3]:
import pandas as pd
import numpy as np
import os

def load_data(sector, ticker, start_date, end_date):
    script_dir = os.path.dirname(os.path.abspath(__file__))  # Get the directory of the script
    file_path = os.path.join(script_dir, "etf_data", f"{sector}_{ticker}_{start_date}_to_{end_date}.csv")
    print(f"Trying to load {file_path}")
    try:
        df = pd.read_csv(file_path, encoding='utf-8')
        return df
    except FileNotFoundError as e:
        print(f"File not found: {file_path}")
        return None

# Main code execution
if __name__ == "__main__":
    script_dir = os.path.dirname(os.path.abspath(__file__))  # Get the directory of the script
    print(f"Current working directory: {script_dir}")
    
    start_date = "2017-01-01"
    end_date = "2022-12-31"
    
    sectors_tickers = {
        "Commodities": ["GLD", "SLV", "PPLT", "PALL","UNG"],
        "Agriculture": ["CORN", "SOYB", "WEAT"],
        "Crude Oil": ["USO", "BNO"],
        "Technology": ["QQQ", "SMH", "HACK","SKYY","BOTZ"],
        "Finance": ["XLF","KBE", "KRE", "KIE"],
        "Healthcare": ["XLV", "XBI", "PJP","IHI"],
        "Market Benchmark": ["SPY"]
        # Add more sectors and their tickers here
    }
    
    
    for sector, tickers in sectors_tickers.items():
        print(f"Processing data for sector: {sector}")
        
        for ticker in tickers:
            print(f"  Processing data for ticker: {ticker}...")
            
            # Load Data
            df = load_data(sector, ticker, start_date, end_date)
            
            # Data Cleaning
            
            # Handling Missing Values
            if df.isnull().any().any():
                df.fillna(method='ffill', inplace=True)
            
            # Handling Outliers
            z_scores = (df - df.mean()) / df.std()
            outliers = (z_scores > 3).any(axis=1)
            df = df[~outliers]
            
            # Data Transformation
            
            # Numerical Transformations: Log Returns
            df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
            
            # Daily Returns
            df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
            
            # Volatility
            df['volatility'] = df['daily_return'].rolling(window=21).std() * np.sqrt(252)
            
            # Momentum
            df['momentum'] = df['Close'] / df['Close'].rolling(window=90).mean() - 1
            
            # Categorical Data: Direction
            df['Direction'] = np.where(df['log_return'] > 0, 1, 0)
            
            # Date-Time Formatting
            df['Date'] = pd.to_datetime(df['Date'])
            
            # Data Resampling: Weekly Average
            weekly_data = df.resample('W', on='Date').mean()
            
            # Feature Engineering: Moving Averages
            df['MA_50'] = df['Close'].rolling(window=50).mean()
            df['MA_100'] = df['Close'].rolling(window=100).mean()
            df['MA_200'] = df['Close'].rolling(window=200).mean()
            
            # Remove first year data to start from "2018-01-01"
            df = df[df['Date'] >= "2018-01-01"]
            
            # Saving pre-processed DataFrame for further steps
            save_path = f"preprocessed_{sector}_{ticker}_2018-01-01_to_{end_date}.csv"
            df.to_csv(save_path, index=False)
            print(f"  Saved processed data to {save_path}")


NameError: name '__file__' is not defined

In [ ]:
print(f"Current working directory: {os.getcwd()}")

In [ ]:
import pandas as pd
import numpy as np
import os

# Function to Load Data
def load_data(sector, ticker, start_date, end_date):
    file_path = f"{sector}_{ticker}_{start_date}_to_{end_date}.xlsx"
    print(f"Trying to load {file_path}")
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError as e:
        print(f"File not found: {file_path}")
        return None

# Main code execution
if __name__ == "__main__":
    print(f"Current working directory: {os.getcwd()}")  # Print the current working directory
    
    start_date = "2017-01-01"
    end_date = "2022-12-31"
    
    sectors_tickers = {
        "Commodities": ["GLD", "SLV", "PPLT", "PALL", "JJC", "JJU", "UNG"],
        "Agriculture": ["CORN", "SOYB", "WEAT", "BAL", "JO"],
        "Crude Oil": ["USO", "BNO"],
        "Technology": ["QQQ", "SMH", "HACK","SKYY","BOTZ"],
        "Finance": ["XLF","KBE", "KRE", "KIE"],
        "Healthcare": ["XLV", "XBI", "PJP","IHI"],
        "Market Benchmark": ["SPY"]
        # Add more sectors and their tickers here
    }
    
    for sector, tickers in sectors_tickers.items():
        print(f"Processing data for sector: {sector}")
        
        for ticker in tickers:
            print(f"  Processing data for ticker: {ticker}...")
            
            # Load Data
            df = load_data(sector, ticker, start_date, end_date)
            
            # Data Cleaning

            #  Only proceed if df is not None
            if df is not None:
                
                # For example: Handling Missing Values
                if df.isnull().any().any():
                    df.fillna(method='ffill', inplace=True)
                    
                # Check for non-numeric values in the dataframe
                non_numeric_columns = df.select_dtypes(exclude=[np.number]).columns
                if not non_numeric_columns.empty:
                    print(f"Non-numeric columns: {non_numeric_columns}")

                non_numeric_rows = df[~df.applymap(np.isreal).all(1)]
                if not non_numeric_rows.empty:
                    print("Rows with non-numeric values:")
                    print(non_numeric_rows)

                # Now calculate z-scores
                z_scores = (df - df.mean()) / df.std()
                
                # Data Transformation
                
                # Numerical Transformations: Log Returns
                df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
                
                # Daily Returns
                df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
                
                # Volatility
                df['volatility'] = df['daily_return'].rolling(window=21).std() * np.sqrt(252)
                
                # Momentum
                df['momentum'] = df['Close'] / df['Close'].rolling(window=90).mean() - 1
                
                # Categorical Data: Direction
                df['Direction'] = np.where(df['log_return'] > 0, 1, 0)
                
                # Date-Time Formatting
                df['Date'] = pd.to_datetime(df['Date'])
                
                # Data Resampling: Weekly Average
                weekly_data = df.resample('W', on='Date').mean()
                
                # Feature Engineering: Moving Averages
                df['MA_50'] = df['Close'].rolling(window=50).mean()
                df['MA_100'] = df['Close'].rolling(window=100).mean()
                df['MA_200'] = df['Close'].rolling(window=200).mean()
                
                # Remove first year data to start from "2018-01-01"
                df = df[df['Date'] >= "2018-01-01"]
                
                # Saving pre-processed DataFrame for further steps
                save_path = f"preprocessed_{sector}_{ticker}_2018-01-01_to_{end_date}.xlsx"
                df.to_csv(save_path, index=False)
                print(f"  Saved processed data to {save_path}")

            else:
                print(f"No data available for sector: {sector}, ticker: {ticker}")

In [2]:
import pandas as pd
import numpy as np
import os

# Function to Load Data
def load_data(sector, ticker, start_date, end_date):
    file_path = f"{sector}_{ticker}_{start_date}_to_{end_date}.xlsx"
    print(f"Trying to load {file_path}")
    try:
        df = pd.read_excel(file_path)
        return df
    except FileNotFoundError as e:
        print(f"File not found: {file_path}")
        return None

# Main code execution
if __name__ == "__main__":
    print(f"Current working directory: {os.getcwd()}")  # Print the current working directory
    
    start_date = "2017-01-01"
    end_date = "2022-12-31"
    
    sectors_tickers = {
        "Commodities": ["GLD", "SLV", "PPLT", "PALL", "JJC", "JJU", "UNG"],
        "Agriculture": ["CORN", "SOYB", "WEAT", "BAL", "JO"],
        "Crude Oil": ["USO", "BNO"],
        "Technology": ["QQQ", "SMH", "HACK","SKYY","BOTZ"],
        "Finance": ["XLF","KBE", "KRE", "KIE"],
        "Healthcare": ["XLV", "XBI", "PJP","IHI"],
        "Market Benchmark": ["SPY"]
        # Add more sectors and their tickers here
    }
    
    for sector, tickers in sectors_tickers.items():
        print(f"Processing data for sector: {sector}")
        
        for ticker in tickers:
            print(f"  Processing data for ticker: {ticker}...")
            
            # Load Data
            df = load_data(sector, ticker, start_date, end_date)
            
            # Data Cleaning

            #  Only proceed if df is not None
            if df is not None:
                
                # For example: Handling Missing Values
                if df.isnull().any().any():
                    df.fillna(method='ffill', inplace=True)
                    
                # Check for non-numeric values in the dataframe
                non-numeric_columns = df.select_dtypes(exclude=[np.number]).columns
                if not non_numeric_columns.empty:
                    print(f"Non-numeric columns: {non_numeric_columns}")

                non_numeric_rows = df[~df.applymap(np.isreal).all(1)]
                if not non_numeric_rows.empty:
                    print("Rows with non-numeric values:")
                    print(non_numeric_rows)

                # Now calculate z-scores
                z_scores = (df - df.mean()) / df.std()
                
                # Data Transformation
                
                # Numerical Transformations: Log Returns
                df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
                
                # Daily Returns
                df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
                
                # Volatility
                df['volatility'] = df['daily_return'].rolling(window=21).std() * np.sqrt(252)
                
                # Momentum
                df['momentum'] = df['Close'] / df['Close'].rolling(window=90).mean() - 1
                
                # Categorical Data: Direction
                df['Direction'] = np.where(df['log_return'] > 0, 1, 0)
                
                # Date-Time Formatting
                df['Date'] = pd.to_datetime(df['Date'])
                
                # Data Resampling: Weekly Average
                weekly_data = df.resample('W', on='Date').mean()
                
                # Feature Engineering: Moving Averages
                df['MA_50'] = df['Close'].rolling(window=50).mean()
                df['MA_100'] = df['Close'].rolling(window=100).mean()
                df['MA_200'] = df['Close'].rolling(window=200).mean()
                
                # Remove first year data to start from "2018-01-01"
                df = df[df['Date'] >= "2018-01-01"]
                
                # Saving pre-processed DataFrame for further steps
                save_path = f"preprocessed_{sector}_{ticker}_2018-01-01_to_{end_date}.xlsx"
                df.to_excel(save_path, index=False)
                print(f"  Saved processed data to {save_path}")

            else:
                print(f"No data available for sector: {sector}, ticker: {ticker}")


SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (1432020779.py, line 53)

In [6]:
import pandas as pd
import numpy as np
import os

# Function to Load Data
def load_data(sector, ticker, start_date, end_date):
    file_path = f"{sector}_{ticker}_{start_date}_to_{end_date}.csv"
    print(f"Trying to load {file_path}")
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError as e:
        print(f"File not found: {file_path}")
        return None

# Main code execution
if __name__ == "__main__":
    print(f"Current working directory: {os.getcwd()}")  # Print the current working directory
    
    start_date = "2017-01-01"
    end_date = "2022-12-31"
    
    sectors_tickers = {
        "Commodities": ["GLD", "SLV", "PPLT", "PALL", "JJC", "JJU", "UNG"],
        "Agriculture": ["CORN", "SOYB", "WEAT", "BAL", "JO"],
        "Crude Oil": ["USO", "BNO"],
        "Technology": ["QQQ", "SMH", "HACK","SKYY","BOTZ"],
        "Finance": ["XLF","KBE", "KRE", "KIE"],
        "Healthcare": ["XLV", "XBI", "PJP","IHI"],
        "Market Benchmark": ["SPY"]
        # Add more sectors and their tickers here
    }
    
    for sector, tickers in sectors_tickers.items():
        print(f"Processing data for sector: {sector}")
        
        for ticker in tickers:
            print(f"  Processing data for ticker: {ticker}...")
            
            # Load Data
            df = load_data(sector, ticker, start_date, end_date)
            
            # Data Cleaning
            
            # Handling Missing Values
            if df.isnull().any().any():
                df.fillna(method='ffill', inplace=True)
            
            # Handling Outliers
            z_scores = (df - df.mean()) / df.std()
            outliers = (z_scores > 3).any(axis=1)
            df = df[~outliers]
            
            # Data Transformation
            
            # Numerical Transformations: Log Returns
            df['log_return'] = np.log(df['Close'] / df['Close'].shift(1))
            
            # Daily Returns
            df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
            
            # Volatility
            df['volatility'] = df['daily_return'].rolling(window=21).std() * np.sqrt(252)
            
            # Momentum
            df['momentum'] = df['Close'] / df['Close'].rolling(window=90).mean() - 1
            
            # Categorical Data: Direction
            df['Direction'] = np.where(df['log_return'] > 0, 1, 0)
            
            # Date-Time Formatting
            df['Date'] = pd.to_datetime(df['Date'])
            
            # Data Resampling: Weekly Average
            weekly_data = df.resample('W', on='Date').mean()
            
            # Feature Engineering: Moving Averages
            df['MA_50'] = df['Close'].rolling(window=50).mean()
            df['MA_100'] = df['Close'].rolling(window=100).mean()
            df['MA_200'] = df['Close'].rolling(window=200).mean()
            
            # Remove first year data to start from "2018-01-01"
            df = df[df['Date'] >= "2018-01-01"]
            
            # Saving pre-processed DataFrame for further steps
            save_path = f"preprocessed_{sector}_{ticker}_2018-01-01_to_{end_date}.csv"
            df.to_csv(save_path, index=False)
            print(f"  Saved processed data to {save_path}")

Current working directory: C:\Users\saide\Documents\Final_Year_Dissertation\ETF_Data\data\raw_etf_data
Processing data for sector: Commodities
  Processing data for ticker: GLD...
Trying to load Commodities_GLD_2017-01-01_to_2022-12-31.csv


TypeError: Could not convert ['2017-01-032017-01-042017-01-052017-01-062017-01-092017-01-102017-01-112017-01-122017-01-132017-01-172017-01-182017-01-192017-01-202017-01-232017-01-242017-01-252017-01-262017-01-272017-01-302017-01-312017-02-012017-02-022017-02-032017-02-062017-02-072017-02-082017-02-092017-02-102017-02-132017-02-142017-02-152017-02-162017-02-172017-02-212017-02-222017-02-232017-02-242017-02-272017-02-282017-03-012017-03-022017-03-032017-03-062017-03-072017-03-082017-03-092017-03-102017-03-132017-03-142017-03-152017-03-162017-03-172017-03-202017-03-212017-03-222017-03-232017-03-242017-03-272017-03-282017-03-292017-03-302017-03-312017-04-032017-04-042017-04-052017-04-062017-04-072017-04-102017-04-112017-04-122017-04-132017-04-172017-04-182017-04-192017-04-202017-04-212017-04-242017-04-252017-04-262017-04-272017-04-282017-05-012017-05-022017-05-032017-05-042017-05-052017-05-082017-05-092017-05-102017-05-112017-05-122017-05-152017-05-162017-05-172017-05-182017-05-192017-05-2